In [ ]:
!pip install darts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
import pandas_gbq as gbq
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, MissingValuesFiller, InvertibleMapper
import sklearn
from darts.models.filtering.moving_average_filter import MovingAverageFilter
from tqdm import tqdm
from darts.models import *
from darts.metrics import mape, smape, mae
from darts import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from sklearn.model_selection import train_test_split
from darts.metrics import *
import joblib


#Read the csv to df1 dataframe
df1=pd.read_excel('rl_emea_dtc_fmt (1).xlsx')
df1['l4_name']=df1['merch_org_desc'] + df1['brand_group_desc'] + df1['plan_level3_name']+df1['prod_class_desc']+ df1['season']
df1=df1[~df1.Dates.isnull()].reset_index(drop=True)


max_date=df1.Dates.max()

input_chunk_length=26
forecast_horizon=26
output_chunk_length=26

tft_model = TFTModel(
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        hidden_size=32,
        lstm_layers=1,
        batch_size=16,
        n_epochs=50,
        dropout=0.1,
        add_encoders={
            'cyclic': {'future': ['month']},
            'datetime_attribute': {'future': ['month']},
            'position': {'past': ['relative'], 'future': ['relative']}
        },
        add_relative_index=False,
        optimizer_kwargs={"lr": 1e-3},
        random_state=42,
    )
tide_model = TiDEModel(input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        hidden_size=32,
        batch_size=16,
        n_epochs=50,
        dropout=0.1,
        add_encoders={
            'cyclic': {'future': ['month']},
            'datetime_attribute': {'future': ['month']},
            'position': {'past': ['relative'], 'future': ['relative']}
        },
        random_state=42,)
nhits_model = NHiTSModel( input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length)
blockrnn_model = BlockRNNModel(input_chunk_length=input_chunk_length, output_chunk_length=output_chunk_length)
models = {
    'TFT': tft_model,
    'Tide': tide_model,
    'Nhits': nhits_model,
    'BlockRNN': blockrnn_model
}

In [ ]:
target_feature='qty'
features=['msrp', 'discount_percentage','trend',
       'fiscal_bi_week','event_impact','qty_lag_26','RollingMin_26','RollingMax_26','SMA_26','covid_flag',
          'season_encoding','month']
models_to_run=['Tide','TFT']
grouping_level='l4_name'
lower_level='l5_name'

train_test_split_date='2022-10-02'

In [ ]:
grouping_level_list=df1.l4_name.unique()
len(grouping_level_list)

126

In [ ]:
def custom_increment_fiscal_year_week(df):
    result = []
    prev_value = None
    increment = 1  # Initial increment value

    for index, row in df.iterrows():
        if pd.isna(row['fiscal_year_week']) or str(row['fiscal_year_week']).lower() == 'nan':
            if prev_value is not None:
                next_value = prev_value + increment
                fiscal_year, week = divmod(next_value, 100)

                # Check if the resulting week is greater than 53 or the week part is '00', and adjust accordingly
                if week > 53 or week == 0:
                    fiscal_year += 1
                    week = 1

                result.append(fiscal_year * 100 + week)
                increment += 1  # Increment the value for each missing row
            else:
                result.append(np.nan)
        else:
            result.append(row['fiscal_year_week'])
            prev_value = row['fiscal_year_week']
            increment = 1  # Reset increment when a non-missing value is encountered

    return pd.Series(result, index=df.index)





def custom_increment(series):
    result = []
    prev_value = None
    increment = 1  # Initial increment value

    for value in series:
        if pd.isna(value):  # Handle missing values
            if prev_value is not None:
                result.append(prev_value + increment)
                increment += 1  # Increment the value for each missing row
            else:
                result.append(np.nan)
        else:
            result.append(value)
            prev_value = value
            increment = 1  # Reset increment when a non-missing value is encountered

    return result

In [ ]:
##########################This can be added in preprocessing#########################################################
static_features=[grouping_level,lower_level]
impute_features_ffill=['msrp','trend','mat_count']
impute_features_zero=['qty','discount_percentage','corpus_christi',
       'midsummer_day_sweden', 'eu_event1', 'eu_event2', 'eu_event3',
       'national_day_france', 'national_day_belgium', 'eu_event4', 'eu_event5',
       'restoration_of_independence_day_portugal', 'immaculate_conception',
       'pre_christmas_week', 'christmas', 'new_year', 'epiphany',
       'post_new_year_week', 'eu_event6', 'chinese_new_year']
impute_features_autoincrement=['age']

imputed_dfs = []
for lower_level_name, lower_level_df in df1.groupby(lower_level):
    # Convert 'Dates' to datetime type
    lower_level_df['Dates'] = pd.to_datetime(lower_level_df['Dates'])

    # Generate a complete date range for the lower_level
    complete_date_range = pd.date_range(start=lower_level_df['Dates'].min(), end=max_date, freq='W')

    # Create a DataFrame with the complete date range
    complete_dates_df = pd.DataFrame({'Dates': complete_date_range})

    # Merge the complete date range with the original data using a left join
    lower_level_df = complete_dates_df.merge(lower_level_df, on='Dates', how='left')

    # Sort the DataFrame by 'Dates'
    lower_level_df.sort_values(by='Dates', inplace=True)

    # Forward fill missing values for impute_features_ffill columns
    lower_level_df[impute_features_ffill] = lower_level_df[impute_features_ffill].ffill().bfill()
    lower_level_df[static_features] = lower_level_df[static_features].ffill().bfill()

    # Fill missing values with 0 for impute_features_zero columns
    lower_level_df[impute_features_zero] = lower_level_df[impute_features_zero].fillna(0)

    # Create a new column 'qty_lag_26' with the lagged values
    lower_level_df['qty_lag_26'] = lower_level_df['qty'].shift(26)
    lower_level_df['RollingMin_26'] = lower_level_df['qty'].rolling(window=26).min()
    lower_level_df['RollingMax_26'] = lower_level_df['qty'].rolling(window=26).max()
    lower_level_df['SMA_26'] = lower_level_df['qty'].rolling(window=26).mean()
    lower_level_df['season_encoding'] = (lower_level_df['Dates'].dt.month % 12 + 3) // 3
    lower_level_df['month'] = lower_level_df['Dates'].dt.month

    # Append the imputed DataFrame to the list
    imputed_dfs.append(lower_level_df)

# Concatenate the imputed DataFrames
df_imputed = pd.concat(imputed_dfs, ignore_index=True)
df_imputed['qty']=df_imputed['qty'].fillna(0)

df_imputed['fiscal_year_week'] = df_imputed.groupby(lower_level, group_keys=False).apply(custom_increment_fiscal_year_week)
df_imputed['fiscal_year_week'] = df_imputed['fiscal_year_week'].astype(int)
df_imputed[['qty_lag_26','RollingMin_26','RollingMax_26','SMA_26']]=df_imputed[['qty_lag_26','RollingMin_26','RollingMax_26','SMA_26']].fillna(0)
# Print the imputed DataFrame
print(df_imputed.shape)

df_imputed['fiscal_year_week'] = df_imputed['fiscal_year_week'].astype(str)
df_imputed['week'] = df_imputed['fiscal_year_week'].str[-2:]
mapping={'01': 1, '02': 1, '03': 2, '04': 2, '05': 3, '06': 3, '07': 4, '08': 4, '09': 5, '10': 5, '11': 6, '12': 6, '13': 7, '14': 7, '15': 8, '16': 8, '17': 9, '18': 9, '19': 10, '20': 10, '21': 11, '22': 11, '23': 12, '24': 12, '25': 13, '26': 13, '27': 14, '28': 14, '29': 15, '30': 15, '31': 16, '32': 16, '33': 17, '34': 17, '35': 18, '36': 18, '37': 19, '38': 19, '39': 20, '40': 20, '41': 21, '42': 21, '43': 22, '44': 22, '45': 23, '46': 23, '47': 24, '48': 24, '49': 25, '50': 25, '51': 26, '52': 26, '53': 26}
# Map the biweeks based on the 'week' column
df_imputed['fiscal_bi_week'] = df_imputed['week'].map(mapping)

(44045, 49)


In [ ]:
df_imputed['Dates'] = pd.to_datetime(df_imputed['Dates'])
event_columns = ['corpus_christi',
       'midsummer_day_sweden', 'eu_event1', 'eu_event2', 'eu_event3',
       'national_day_france', 'national_day_belgium', 'eu_event4', 'eu_event5',
       'restoration_of_independence_day_portugal', 'immaculate_conception',
       'pre_christmas_week', 'christmas', 'new_year', 'epiphany',
       'post_new_year_week', 'eu_event6', 'chinese_new_year']
df_imputed['event_impact'] = df_imputed[event_columns].sum(axis=1)

# Assuming df is your DataFrame
df_imputed['fiscal_year_week'] = df_imputed['fiscal_year_week'].astype(str)  # Convert to string

df_imputed['covid_flag'] = df_imputed['fiscal_year_week'].str.startswith('2021').astype(int)

# Convert the 'fiscal_year_week' column back to int if needed
df_imputed['fiscal_year_week'] = df_imputed['fiscal_year_week'].astype(int)

In [ ]:
model_dict={}
prod_dict={}
test_all=pd.DataFrame()
predictions_all=pd.DataFrame()

import time
start = time.process_time()

# Loop through the models
for model_name in models_to_run:
    print(f"Evaluating model: {model_name}")
    for i in grouping_level_list:
      print(i)
      # Create an empty dictionary to store product-specific TimeSeries
      feature_scaler = Scaler()
      target_scaler = Scaler()
      feature_validation_scaler = Scaler()
      target_validation_scaler = Scaler()
      product_list=[]
      product_series_dict = {}
      train_series=[]
      train_series_scaled=[]
      past_covariates_scaled=[]
      new_series=[]
      past_covariates=[]
      product_list_validation=[]
      past_covariates_validation=[]
      df2=df_imputed[df_imputed[grouping_level]==i].reset_index(drop=True)
      print('number of l5',df2.l5_name.nunique())
      # Iterate through unique product codes
      for code in df2[lower_level].unique():
        # Filter the DataFrame for the current lower_level

        lower_level_df = df2[df2[lower_level] == code].reset_index(drop=True)
        lower_level_df.sort_values(by='Dates', inplace=True)
        lower_level_df.fillna(0,inplace=True)
        train_df=lower_level_df[lower_level_df.Dates < train_test_split_date].reset_index(drop=True)
        train_df.sort_values(by='Dates', inplace=True)
        #train_df_target=train_df.iloc[:-output_chunk_length]
        test_df=lower_level_df[lower_level_df.Dates >= train_test_split_date].reset_index(drop=True)
        test_df.sort_values(by='Dates', inplace=True)

        # Check if the series is long enough
        if len(train_df) < (input_chunk_length + output_chunk_length):
              print(f"Skipping {lower_level} {code} due to insufficient data points.")
              continue

        target_series = TimeSeries.from_dataframe(train_df[['qty','Dates']],time_col='Dates',value_cols='qty',fill_missing_dates=True,fillna_value=0, freq='W')
        # Create covariate series (msrp, percent_off, store_count)
        covariates = TimeSeries.from_dataframe(train_df[features + ['Dates']],time_col='Dates',fill_missing_dates=True, freq='W')


        # Create additional time-based features
        year = datetime_attribute_timeseries(time_index=covariates.time_index, attribute="year")
        month = datetime_attribute_timeseries(time_index=covariates.time_index, attribute="month")
        weekofyear = datetime_attribute_timeseries(time_index=covariates.time_index, attribute="weekofyear")
        linear_trend = TimeSeries.from_times_and_values(
            times=covariates.time_index,
            values=np.arange(len(covariates)),
            columns=["linear_trend"]
        )

        covariate_series=covariates.stack(year).stack(month).stack(weekofyear).stack(linear_trend)

        # Store the target and covariate series in the dictionary
        product_series_dict[code] = (target_series, covariate_series)
        train_series.append(product_series_dict[code][0])
        past_covariates.append(product_series_dict[code][1])
        product_list.append(code)
        if len(test_df) >= forecast_horizon:
          test_df['actual_qty']=test_df['qty']
          test_df['qty']=0
          train_df1=train_df.tail(input_chunk_length + output_chunk_length)
          if len(train_df1) < (input_chunk_length + output_chunk_length):
            continue
          new_df=pd.concat([train_df1,test_df],axis=0, ignore_index=True)
          new_df.fillna(0,inplace=True)
          target_series_validation = TimeSeries.from_dataframe(train_df1[['qty','Dates']].reset_index(drop=True),time_col='Dates',value_cols='qty',fill_missing_dates=True,fillna_value=0, freq='W')
          # Create covariate series (msrp, percent_off, store_count)
          covariates_validation = TimeSeries.from_dataframe(new_df[features + ['Dates']].reset_index(drop=True),time_col='Dates',fill_missing_dates=True, freq='W')
          # covariates= covariates.pd_dataframe().fillna(method='ffill').dropna()
          # covariates = TimeSeries.from_dataframe(covariates)

          # Create additional time-based features
          year = datetime_attribute_timeseries(time_index=covariates_validation.time_index, attribute="year")
          month = datetime_attribute_timeseries(time_index=covariates_validation.time_index, attribute="month")
          weekofyear = datetime_attribute_timeseries(time_index=covariates_validation.time_index, attribute="weekofyear")
          linear_trend = TimeSeries.from_times_and_values(
              times=covariates_validation.time_index,
              values=np.arange(len(covariates_validation)),
              columns=["linear_trend"]
          )

          covariate_series_validation=covariates_validation.stack(year).stack(month).stack(weekofyear).stack(linear_trend)
          new_series.append(target_series_validation)
          past_covariates_validation.append(covariate_series_validation)
          product_list_validation.append(code)
          test_all=pd.concat([test_all,test_df],ignore_index=True)

      train_series_scaled=target_scaler.fit_transform(train_series)
      past_covariates_scaled=feature_scaler.fit_transform(past_covariates)
      new_series_scaled=target_validation_scaler.fit_transform(new_series)
      past_covariates_validation_scaled=feature_validation_scaler.fit_transform(past_covariates_validation)
      model = models[model_name]
      if len(train_series)<1:
        continue
      model.fit(train_series_scaled, past_covariates=past_covariates_scaled)
      #model.fit(train_series, past_covariates=past_covariates)
      prod_dict[i]=product_list_validation
      if len(new_series) < 1:
        continue
      pred=model.predict(n=forecast_horizon, series=new_series_scaled, past_covariates=past_covariates_validation_scaled)
      predictions=target_validation_scaler.inverse_transform(pred)
      #predictions=model.predict(n=forecast_horizon, series=new_series, past_covariates=past_covariates_validation)
      for k in range(0,len(predictions)):
        temp=predictions[k].pd_dataframe().reset_index()
        temp[lower_level]=prod_dict[i][k]
        temp[grouping_level]=i
        temp['model_name']=model_name
        temp.rename(columns = {'qty':'predicted_qty'}, inplace = True)
        predictions_all=pd.concat([temp,predictions_all],ignore_index=True)

test_all['Dates'] = pd.to_datetime(test_all['Dates'])
result=pd.merge(test_all,predictions_all,on=[lower_level,'Dates',grouping_level],how='inner')
result=result.drop_duplicates()
result['predicted_qty']=np.where(result['predicted_qty']<0,0,result['predicted_qty'])
res=result[~result.predicted_qty.isnull()].reset_index(drop=True)
result1=res.groupby([grouping_level,lower_level,'model_name']).agg({'actual_qty':'sum','predicted_qty':'sum'}).reset_index()
result1['residual']=abs(result1['actual_qty']-result1['predicted_qty'])
result1['wape']=result1['residual']/result1['actual_qty']
best_models = result1.groupby(lower_level)['wape'].idxmin()
# Create a new dataframe with the best model for each family
best_df = result1.loc[best_models].reset_index(drop=True)
result1.to_csv('result_rl.csv')
best_df.to_csv('best_df_rl.csv')
predictions_final=pd.merge(result,best_df[['l5_name','model_name']],on=['l5_name','model_name'],how='inner')
predictions_final.to_csv('predictions_all_rl.csv')
del model
print('Time taken',time.process_time() - start)

Evaluating model: Tide
WOMENS APPARELLAUREN RTWW APPARELA LINEFALL
number of l5 1


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                  | Type             | Params
-----------------------------------------------------------
0 | criterion             | MSELoss          | 0     
1 | train_metrics         | MetricCollection | 0     
2 | val_metrics           | MetricCollection | 0     
3 | past_cov_projection   | _ResidualBlock   | 780   
4 | future_cov_projection | _ResidualBlock   | 312   
5 | encoders              | Sequential       | 22.8 K
6 | decoders              | Sequential       | 28.5 K
7 | temporal_decoder      | _ResidualBlock   | 726   

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

...

WOMENS APPARELLIC POLO RALPH LAURENW APPAREL1 PC SWIMPRE-SPRING
number of l5 1


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                  | Type             | Params
-----------------------------------------------------------
0 | criterion             | MSELoss          | 0     
1 | train_metrics         | MetricCollection | 0     
2 | val_metrics           | MetricCollection | 0     
3 | past_cov_projection   | _ResidualBlock   | 780   
4 | future_cov_projection | _ResidualBlock   | 312   
5 | encoders              | Sequential       | 22.8 K
6 | decoders              | Sequential       | 28.5 K
7 | temporal_decoder      | _ResidualBlock   | 726   

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Buffered data was truncated after reaching the output size limit.

In [ ]:
best_df

,l4_name,l5_name,model_name,actual_qty,predicted_qty,residual,wape
0,WOMENS APPARELLAUREN DRESSESW APPARELCOCKTAIL ...,LAUREN DRESSESWOMENS APPARELAPPARELW APPARELDR...,Tide,1463.0,741.771152,721.228848,0.492979
1,WOMENS APPARELLAUREN DRESSESW APPARELCOCKTAIL ...,LAUREN DRESSESWOMENS APPARELAPPARELW APPARELDR...,TFT,142.0,335.309076,193.309076,1.361332
2,WOMENS APPARELLAUREN DRESSESW APPARELCOCKTAIL ...,LAUREN DRESSESWOMENS APPARELAPPARELW APPARELDR...,TFT,951.0,333.578100,617.421900,0.649234
3,WOMENS APPARELLAUREN DRESSESW APPARELCOCKTAIL ...,LAUREN DRESSESWOMENS APPARELAPPARELW APPARELDR...,TFT,265.0,523.867101,258.867101,0.976857
4,WOMENS APPARELLAUREN DRESSESW APPARELDAY DRESS...,LAUREN DRESSESWOMENS APPARELAPPARELW APPARELDR...,Tide,1030.0,756.436091,273.563909,0.265596
...,...,...,...,...,...,...,...
179,WOMENS APPARELRL COLLECTIONW APPARELT-SHIRTSPRING,RL COLLECTIONWOMENS APPARELAPPARELW APPARELKNI...,TFT,300.0,207.566522,92.433478,0.308112
180,WOMENS APPARELRL COLLECTIONW APPARELBUTTON FRO...,RL COLLECTIONWOMENS APPARELAPPARELW APPARELSHI...,Tide,320.0,341.654924,21.654924,0.067672
181,WOMENS APPARELRL COLLECTIONW APPARELBUTTON FRO...,RL COLLECTIONWOMENS APPARELAPPARELW APPARELSHI...,TFT,380.0,364.107230,15.892770,0.041823
182,WOMENS APPARELRL COLLECTIONW APPARELPULLOVERFALL,RL COLLECTIONWOMENS APPARELAPPARELW APPARELSWE...,Tide,1251.0,1053.861829,197.138171,0.157584


In [ ]:
best_df

,l4_name,l5_name,model_name,actual_qty,predicted_qty,residual,wape
0,WOMENS APPARELLIC LAUREN RTWW APPARELCOATFALL,LIC LAUREN RTWWOMENS APPARELAPPARELW APPARELOU...,Tide,12960.0,9986.352521,2973.647479,0.229448
1,WOMENS APPARELLIC LAUREN RTWW APPARELCOATFALL,LIC LAUREN RTWWOMENS APPARELAPPARELW APPARELOU...,Tide,12744.0,4144.835624,8599.164376,0.674762
2,WOMENS APPARELLIC LAUREN RTWW APPARELCOATFALL,LIC LAUREN RTWWOMENS APPARELAPPARELW APPARELOU...,Tide,466.0,236.821605,229.178395,0.491799


In [ ]:
best_df

,l4_name,l5_name,model_name,actual_qty,predicted_qty,residual,wape
0,WOMENS APPARELLIC LAUREN RTWW APPARELCOATFALL,LIC LAUREN RTWWOMENS APPARELAPPARELW APPARELOU...,TFT,12960.0,11888.407309,1071.592691,0.082685
1,WOMENS APPARELLIC LAUREN RTWW APPARELCOATFALL,LIC LAUREN RTWWOMENS APPARELAPPARELW APPARELOU...,TFT,12744.0,4688.518826,8055.481174,0.632100
2,WOMENS APPARELLIC LAUREN RTWW APPARELCOATFALL,LIC LAUREN RTWWOMENS APPARELAPPARELW APPARELOU...,TFT,466.0,239.112403,226.887597,0.486883


In [ ]:
#Tide only grouping and lower
result1